## Reference:
- https://learn.deeplearning.ai/courses/ai-agents-in-langgraph/lesson/

# Overview: Persistence and Streaming
This notebook goes over 2 topics
1. **Streaming**:
    - Lets us see what the agent is doing. This can be done at 2 levels:
        - **Streaming messages**: (AI message, Tool message, next AI message, etc.)
            - Done through `graph.stream()` method
        - **Streaming tokens**: (E.g. LLM tokens)
            - Done through `graph.astream_events()` method (which are `async` methods on all LG/LC objects).
2. **Persistence**:
    - Lets keep the state of an agent. 
    - We can go back & resume from that state for long running applications or multiple conversations. This is done using:
        - **Checkpointer** concept: 
            - Checkpoints the state after & between every node. 
            - It can be in memory / persistent db.
            - Pass it to `graph.compile(checkpointer=<>)`
            - We need regular checkpointer (e.g. built in db `SqlliteSaver`) for **Streaming messages** and `async` checkpointer (e.g. `AsyncSqliteSaver`) for **Streaming tokens**.
        - **Thread config** conecpt:
            - Allows to keep track of different threads in checkpointer & have multiple conversations at same time. 
            - Should be passed to `graph.stream()/astream_events()` methods.


In [1]:
from dotenv import load_dotenv

_ = load_dotenv()

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [3]:
tool = TavilySearchResults(max_results=2)

In [4]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

## Persistence & Streaming messages 

In [5]:
# Persistence: Checkpointer
from langgraph.checkpoint.sqlite import SqliteSaver
memory = SqliteSaver.from_conn_string(":memory:") 
# In memory db. Not persistent. But can be connected to external db

In [6]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [7]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-4o")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [8]:
messages = [HumanMessage(content="What is the weather in sf?")]

In [9]:
# Need it to track multiple conversations in checkpointer 
thread = {"configurable": {"thread_id": "1"}}

In [10]:
# Calling stream instead of invoke
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_bmfLa92f6oAIKN9KvXtqbKDz', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 151, 'total_tokens': 173, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_831e067d82', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c76b681d-aa2e-4298-a68a-7bd6679c653e-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_bmfLa92f6oAIKN9KvXtqbKDz'}])]
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_bmfLa92f6oAIKN9KvXtqbKDz'}
Back t

In [11]:
# Continues the context (persistence from checkpoint & thread id)
messages = [HumanMessage(content="What about in la?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_H7uUaTSURmaSKyYnK7YqnvEd', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 1380, 'total_tokens': 1402, 'prompt_tokens_details': {'cached_tokens': 1280, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_f33640a400', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c0fcc71d-1cf8-4900-b69c-edb7265a905b-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_H7uUaTSURmaSKyYnK7YqnvEd'}])]}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_H7uUaTSURmaSKyYnK7Yq

In [12]:
# Continues the context (persistence from checkpoint & thread id)
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content="Currently, both San Francisco and Los Angeles have similar temperatures around 75°F. However, Los Angeles is expected to be slightly warmer during the night with a low of 59°F compared to San Francisco's expected low of 55°F. So, Los Angeles is a bit warmer overall.", response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 2673, 'total_tokens': 2732, 'prompt_tokens_details': {'cached_tokens': 2560, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_f33640a400', 'finish_reason': 'stop', 'logprobs': None}, id='run-f73b5d42-828b-43d2-bc5f-af1abce18784-0')]}


In [13]:
# Breaks the context (different thread id)
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content="Could you please clarify what you're comparing to determine which is warmer? Are you comparing two specific locations, types of clothing, materials, or something else? Let me know so I can provide the appropriate information.", response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 149, 'total_tokens': 192, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_f9f4fb6dbf', 'finish_reason': 'stop', 'logprobs': None}, id='run-0d52c758-ff73-4521-b59f-5e7ecc9e9b75-0')]}


## Streaming tokens

In [14]:
from langgraph.checkpoint.aiosqlite import AsyncSqliteSaver

memory = AsyncSqliteSaver.from_conn_string(":memory:")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [16]:
messages = [HumanMessage(content="What is the weather in SF?")]
thread = {"configurable": {"thread_id": "4"}}
async for event in abot.graph.astream_events({"messages": messages}, thread, version="v1"):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco October 16 2023'}, 'id': 'call_jY6Y9t45b6KvrazqsTXp4Yyy'}
Back to the model!
The| current| weather| in| San| Francisco| for| October| |16|,| |202|3|,| is| expected| to| be| sunny| with| temperatures| around| |72|°F| during| the| day| and| dropping| to| about| |61|°F| at| night|.| There| might| be| some| fog| late| in| the| evening|.| The| wind| is| coming| from| the| west| at| about| |13|.|6| mph|.| Overall|,| it's| a| pleasant| day| with| clear| skies|.|